In [1]:
# !pip install -U segmentation-models-pytorch albumentations --user
import torch
import numpy
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import torchvision.models.segmentation
from torchvision.transforms import ToTensor, ToPILImage
from torch.utils.data import Dataset, DataLoader
import cv2

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import os

In [3]:
import os
import pandas as pd
import random
import json
import math
from data import get_data
from tqdm import tqdm
from torchvision.io import read_image
from sklearn.cluster import DBSCAN, KMeans
import numpy as np
import torch.optim as optim
import pickle
from IPython.display import clear_output

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

2023-12-10 19:01:47.868916: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-10 19:01:47.898311: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-10 19:01:47.898341: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-10 19:01:47.899305: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-10 19:01:47.904631: I tensorflow/core/platform/cpu_feature_guar

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


2023-12-10 18:57:19.897064: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [5]:
data = get_data()

100%|███████████████████████████████████████████| 27/27 [14:45<00:00, 32.81s/it]


In [6]:
def downsample_block(x, n_filters):
    x = layers.Conv2D(n_filters, 3, padding = "same", activation = "elu", kernel_initializer = "he_normal")(x)
    f = layers.Conv2D(n_filters, 3, padding = "same", activation = "elu", kernel_initializer = "he_normal")(x)
    p = layers.MaxPool2D(2)(f)
    p = layers.Dropout(0.3)(p)
    return f, p

In [7]:
def upsample_block(x, conv_features, n_filters):
    x = layers.Conv2DTranspose(n_filters, 3, 2, padding="same")(x)
    x = layers.Dropout(0.3)(x)
    x = layers.concatenate([x, conv_features])
    x = layers.Conv2D(n_filters, 3, padding = "same", activation = "elu", kernel_initializer = "he_normal")(x)
    x = layers.Conv2D(n_filters, 3, padding = "same", activation = "elu", kernel_initializer = "he_normal")(x)
    return x

In [8]:
# https://stackoverflow.com/a/49290815
def jaccard_distance(y_true, y_pred, smooth=100):
    """ Calculates mean of Jaccard distance as a loss function """
    intersection = tf.reduce_sum(y_true * y_pred, axis=(1,2))
    sum_ = tf.reduce_sum(y_true + y_pred, axis=(1,2))
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    jd =  (1 - jac) * smooth
    return tf.reduce_mean(jd)

In [9]:
inputs = layers.Input(shape=(128,128,3))
f1, p1 = downsample_block(inputs, 16)
f2, p2 = downsample_block(p1, 32)
x = layers.Conv2D(64, 3, padding = "same", activation = "elu", kernel_initializer = "he_normal")(p2)
x = layers.Dropout(0.3)(x)
u2 = upsample_block(x, f2, 32)
u1 = upsample_block(u2, f1, 16)
outputs = layers.Conv2D(1, 3, padding = "same", activation = "sigmoid", kernel_initializer = "he_normal")(u1)
unet_model = tf.keras.Model(inputs, outputs, name="U-Net")


2023-12-10 19:16:57.466818: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [10]:
unet_model.summary()

Model: "U-Net"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 128, 128, 16)         448       ['input_1[0][0]']             
                                                                                                  
 conv2d_1 (Conv2D)           (None, 128, 128, 16)         2320      ['conv2d[0][0]']              
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 64, 64, 16)           0         ['conv2d_1[0][0]']            
 D)                                                                                           

In [11]:
unet_model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss='binary_crossentropy',
                  metrics=["accuracy", jaccard_distance])

In [12]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, draft_data, sz, batch_size=16, dim=(128,128), n_channels=3,
                 shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.draft = draft_data
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.len = sz
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(self.len / self.batch_size)

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [k for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(self.len)
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size, *self.dim), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            elem = self.draft[ID]
            # Store sample
            X[i,] = elem[0]

            # Store class
            y[i] = elem[1]

        return X, y

In [13]:
def create_data(sz):
    X = []
    y = []
    for i in range(sz):
        elem = data[i]
        X.append(elem[0])
        y.append(elem[1])
    res = tf.data.Dataset.from_tensors((X, y))
    del X
    del y
    return res
    
train = DataGenerator(data, 5000)
val = DataGenerator(data, 64)
# del data

In [14]:
callbacks = [
    keras.callbacks.ReduceLROnPlateau(),
    keras.callbacks.TensorBoard()
]

In [17]:
model_history = unet_model.fit(train,
                               verbose=2,
                               epochs=10,
                               validation_data=val                                )

Epoch 1/10
312/312 - 51s - loss: 0.2227 - accuracy: 0.9103 - jaccard_distance: 46.9431 - val_loss: 0.2787 - val_accuracy: 0.9008 - val_jaccard_distance: 47.6330 - 51s/epoch - 163ms/step
Epoch 2/10
312/312 - 56s - loss: 0.1958 - accuracy: 0.9217 - jaccard_distance: 42.6258 - val_loss: 0.1738 - val_accuracy: 0.9308 - val_jaccard_distance: 28.9028 - 56s/epoch - 178ms/step
Epoch 3/10
312/312 - 55s - loss: 0.1831 - accuracy: 0.9276 - jaccard_distance: 41.4548 - val_loss: 0.1532 - val_accuracy: 0.9384 - val_jaccard_distance: 41.3530 - 55s/epoch - 177ms/step
Epoch 4/10
312/312 - 56s - loss: 0.1720 - accuracy: 0.9336 - jaccard_distance: 39.6586 - val_loss: 0.1611 - val_accuracy: 0.9394 - val_jaccard_distance: 28.3026 - 56s/epoch - 180ms/step
Epoch 5/10
312/312 - 56s - loss: 0.1693 - accuracy: 0.9349 - jaccard_distance: 39.4981 - val_loss: 0.1425 - val_accuracy: 0.9448 - val_jaccard_distance: 32.1666 - 56s/epoch - 179ms/step
Epoch 6/10
312/312 - 56s - loss: 0.1552 - accuracy: 0.9412 - jaccard_d

In [18]:
fig = plt.figure(figsize=(100, 100))
for i in range(5):
    X, y = val[0]#     X = X.permute(1, 2, 0).numpy()
    X = X[i]
    y = y[i]
    print(X.shape)
    print(y.shape)
    pred = unet_model.predict(np.array([X]))
    fig.add_subplot(5, 3, 3 * i + 1)
    plt.imshow(X)
    fig.add_subplot(5, 3, 3 * i + 2)
    plt.imshow(y * 255, cmap='gray')
    fig.add_subplot(5, 3, 3 * i + 3)
    plt.imshow(pred[0], cmap='gray')
plt.show()

(128, 128, 3)
(128, 128)
1/1 [==============================] - 0s 22ms/step
(128, 128, 3)
(128, 128)
1/1 [==============================] - 0s 20ms/step
(128, 128, 3)
(128, 128)
1/1 [==============================] - 0s 21ms/step
(128, 128, 3)
(128, 128)
1/1 [==============================] - 0s 22ms/step
(128, 128, 3)
(128, 128)
1/1 [==============================] - 0s 22ms/step


In [19]:
unet_model.save_weights('./checkpoints/my_checkpoint_2')